# Install necessary libs, if the computer don't have these (uncomment necessary lib)


In [1]:
# !pip install geocoder
# !pip install geopy
# !pip install folium

In [2]:
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import folium
from folium.plugins import FastMarkerCluster
import mysql.connector
import timeit

***
* chronometer

In [3]:
def check_time(start, end):
    seg = end - start
    dias = seg // 86400
    Seg_Restantes_Após_Dias = seg % 86400
    horas = Seg_Restantes_Após_Dias // 3600         # Divisão inteira
    Seg_Restantes_Apos_Hora = seg % 3600            # Resto da divisão por 3600
    minutos = Seg_Restantes_Apos_Hora // 60         # Divisão inteira dos segundos restantes
    Seg_Restantes_Apos_Minutos = seg % 60
    #print(minutos,'minutos','e', '%.2f' % Seg_Restantes_Apos_Minutos,'segundos')
    print(dias,'dias', horas,'horas', minutos,'minutos','e', '%.2f' % Seg_Restantes_Apos_Minutos,'segundos')

***
* Arquivo gerado a partir do KML do google earth

In [4]:
coord = pd.read_csv('../inputs/dutra_all.csv', delimiter=';')
coord

,Latitude,Longitude
0,-23.525513,-46.587819
1,-23.525513,-46.587481
2,-23.525083,-46.587295
3,-23.524845,-46.586983
4,-23.524453,-46.586675
...,...,...
9061,-22.816778,-43.323532
9062,-22.816855,-43.323396
9063,-22.816929,-43.323227
9064,-22.817023,-43.323105


***
# Divisor de kms

Gera as coordenadas que o arquivo KML gerou pela quantidade de KMs da rodovia. Caso o resultado da divisão não seja exata, é feito o "arredondamento" para que no final cada KM receba as coordenadas de forma correta.

In [5]:
km_total = 403                                        # qual kms total da rodovia (inserir manualmente)
points_total = coord.shape[0]                         # total de pontos de coordenadas vindos do arquivo kml

if points_total % km_total == 0:                      # se a divião for exata
    div_fact = points_total // km_total               # fator de divisão
    print(div_fact)
else:
    div_fact_exato = km_total * (points_total // km_total)   # arredonda a divisor
    remove_n = points_total - div_fact_exato          # linhas a serem removidas
    drop_rows = np.random.choice(coord.index, remove_n, replace=False)  # linhas escolhidas aleatoriamente para serem removidas, a fim de arredondar a diferença
    coord = coord.drop(drop_rows)                     # remoção das linhas escolhidas
    coord = coord.reset_index(drop=True)              # reseta o index
    points_total_changed = coord.shape[0]             # pega o tamanho do dataframe
    div_fact = points_total_changed / km_total        # fator de divisão
    div_fact = int(div_fact)                          # transforma em inteiro


In [6]:
list_lat = pd.DataFrame(['%06.3f' % (coord.loc[i][0]) for i in range(0, len(coord), div_fact)], columns=['Latitude'])

In [7]:
list_lon = pd.DataFrame(['%06.3f' % (coord.loc[i][1]) for i in range(0, len(coord), div_fact)], columns=['Longitude'])

In [8]:
coord = pd.concat([list_lat, list_lon], axis=1)
coord

,Latitude,Longitude
0,-23.526,-46.588
1,-23.518,-46.581
2,-23.508,-46.571
3,-23.501,-46.562
4,-23.496,-46.556
...,...,...
398,-22.797,-43.359
399,-22.802,-43.351
400,-22.806,-43.343
401,-22.809,-43.338


***
# Map

Plota no mapa as coordenadas geradas

In [9]:
map1 = folium.Map(location=[-22.8923, -45.2887],
                  tiles='cartodbpositron',
                  zoom_start=9,
                  )

In [10]:
coord.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]]).add_to(map1), axis=1)

0      <folium.vector_layers.CircleMarker object at 0...
1      <folium.vector_layers.CircleMarker object at 0...
2      <folium.vector_layers.CircleMarker object at 0...
3      <folium.vector_layers.CircleMarker object at 0...
4      <folium.vector_layers.CircleMarker object at 0...
                             ...                        
398    <folium.vector_layers.CircleMarker object at 0...
399    <folium.vector_layers.CircleMarker object at 0...
400    <folium.vector_layers.CircleMarker object at 0...
401    <folium.vector_layers.CircleMarker object at 0...
402    <folium.vector_layers.CircleMarker object at 0...
Length: 403, dtype: object

In [11]:
# map1

***

* Addresses

Para cada coordenada é feita a junção da descrição do KM correspondente

In [12]:
addresses = pd.read_csv('../inputs/dutra.csv', delimiter=';')
addresses

,Rodovia,KM,State
0,Presidente Dutra,232,SP
1,Presidente Dutra,231,SP
2,Presidente Dutra,230,SP
3,Presidente Dutra,229,SP
4,Presidente Dutra,228,SP
...,...,...,...
398,Presidente Dutra,168,RJ
399,Presidente Dutra,167,RJ
400,Presidente Dutra,166,RJ
401,Presidente Dutra,165,RJ


In [13]:
addresses['Latitude'] = ""
addresses['Longitude'] = ""

In [14]:
for x in range(len(addresses)):
    addresses['Latitude'][x] = coord['Latitude'][x]
    addresses['Longitude'][x] = coord['Longitude'][x]

# print(addresses.to_string())
addresses

<ipython-input-14-0912f956dd8b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses['Latitude'][x] = coord['Latitude'][x]
<ipython-input-14-0912f956dd8b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses['Longitude'][x] = coord['Longitude'][x]


,Rodovia,KM,State,Latitude,Longitude
0,Presidente Dutra,232,SP,-23.526,-46.588
1,Presidente Dutra,231,SP,-23.518,-46.581
2,Presidente Dutra,230,SP,-23.508,-46.571
3,Presidente Dutra,229,SP,-23.501,-46.562
4,Presidente Dutra,228,SP,-23.496,-46.556
...,...,...,...,...,...
398,Presidente Dutra,168,RJ,-22.797,-43.359
399,Presidente Dutra,167,RJ,-22.802,-43.351
400,Presidente Dutra,166,RJ,-22.806,-43.343
401,Presidente Dutra,165,RJ,-22.809,-43.338


***
# Gerador de TABLE usando INSERT - ATIVAR MODULOS APENAS QUANDO FOR CRIAR TABLE NA BASE DE DADOS

* Create Table

* Delete Table - opicional

* MySQL connector

* Insert values into Table

* Close Connections

***
# Query, extrair leituras por KM - ATIVAR MODULOS APENAS QUANDO FOR EXTRAIR DADOS DA BASE

* MySQL connector

* Query para extrair as leituras, po KM

* Close Connections

* Export to CSV

***
# Mesclar dados importados com leituras com dados sem leituras

* Kms com leituras

In [15]:
df_dutra_kms = pd.read_csv('../outputs/leituras_dutra.csv', delimiter=',')
df_dutra_kms.head()
# print(df_dutra_kms.to_string())

,Id,state,km,esp_00h_06hs,esp_06_12hs,esp_12_18,esp_18_24hs
0,1,SP,232,1,2,1,1
1,2,SP,231,1,0,0,0
2,3,SP,230,0,1,3,0
3,5,SP,228,2,2,2,7
4,6,SP,227,1,0,1,1


* Kms sem leituras

In [16]:
ls_empyt_dutra_kms = [] # lista de kms sem leituras
index_diff = addresses.index.difference(df_dutra_kms.Id) # extrair kms sem dados de leitura
ls_empyt_dutra_kms.append(index_diff) # adiciona na lista
df_empyt_dutra_kms = pd.DataFrame(ls_empyt_dutra_kms[0], columns = ["Id"])
df_empyt_dutra_kms = df_empyt_dutra_kms.drop([0])

In [17]:
df_empyt_dutra_kms.head()

,Id
1,4
2,13
3,14
4,15
5,16


* Criar colunas com 0 leituras

In [18]:
df_empyt_dutra_kms['esp_00h_06hs'] = "0"
df_empyt_dutra_kms['esp_06_12hs'] = "0"
df_empyt_dutra_kms['esp_12_18'] = "0"
df_empyt_dutra_kms['esp_18_24hs'] = "0"

In [27]:
df_empyt_dutra_kms

,Id,esp_00h_06hs,esp_06_12hs,esp_12_18,esp_18_24hs
1,4,0,0,0,0
2,13,0,0,0,0
3,14,0,0,0,0
4,15,0,0,0,0
5,16,0,0,0,0
...,...,...,...,...,...
336,395,0,0,0,0
337,396,0,0,0,0
338,397,0,0,0,0
339,398,0,0,0,0


* Concatenando colunas

In [20]:
df_dutra_kms_total = pd.concat([df_dutra_kms[['Id', 'esp_00h_06hs', 'esp_06_12hs', 'esp_12_18', 'esp_18_24hs']], \
                                df_empyt_dutra_kms[['Id', 'esp_00h_06hs', 'esp_06_12hs', 'esp_12_18', 'esp_18_24hs']]])\
                               .sort_values('Id').reset_index(drop=True)

df_dutra_kms_total['state'] = addresses['State']
df_dutra_kms_total['km'] = addresses['KM']

In [26]:
df_dutra_kms_total

,Id,esp_00h_06hs,esp_06_12hs,esp_12_18,esp_18_24hs,state,km
0,1,1,2,1,1,SP,232
1,2,1,0,0,0,SP,231
2,3,0,1,3,0,SP,230
3,4,0,0,0,0,SP,229
4,5,2,2,2,7,SP,228
...,...,...,...,...,...,...,...
397,398,0,0,0,0,RJ,169
398,399,0,1,1,0,RJ,168
399,400,1,4,2,3,RJ,167
400,401,1,0,1,1,RJ,166


* Ordenando colunas

In [22]:
cols = list(df_dutra_kms_total.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('state')) #Remove b from list
cols.pop(cols.index('km')) #Remove x from list
cols.pop(cols.index('esp_00h_06hs')) #Remove x from list
cols.pop(cols.index('esp_06_12hs')) #Remove x from list
cols.pop(cols.index('esp_12_18')) #Remove x from list
cols.pop(cols.index('esp_18_24hs')) #Remove x from list
df = df_dutra_kms_total[cols+['state','km', 'esp_00h_06hs', 'esp_06_12hs', 'esp_12_18', 'esp_18_24hs']]

In [29]:
df
# print(df.to_string())

,Id,state,km,esp_00h_06hs,esp_06_12hs,esp_12_18,esp_18_24hs
0,1,SP,232,1,2,1,1
1,2,SP,231,1,0,0,0
2,3,SP,230,0,1,3,0
3,4,SP,229,0,0,0,0
4,5,SP,228,2,2,2,7
...,...,...,...,...,...,...,...
397,398,RJ,169,0,0,0,0
398,399,RJ,168,0,1,1,0
399,400,RJ,167,1,4,2,3
400,401,RJ,166,1,0,1,1


***
* Export indexturas por KM to CSV

In [34]:
df_dutra_kms_out = pd.DataFrame({
                                 'Id'                       : df['Id'].astype(str),
                                 'Rodovia'                  : addresses['Rodovia'][0:-1],
                                 'State'                    : df['state'],
                                 'KM'                       : df['km'].astype(str),
                                 'esp_00h_06hs'             : df['esp_00h_06hs'].astype(str),
                                 'esp_06_12hs'              : df['esp_06_12hs'].astype(str),
                                 'esp_12_18'                : df['esp_12_18'].astype(str),
                                 'esp_18_24hs'              : df['esp_18_24hs'].astype(str)
                               })
# df_dutra_kms_out.to_csv('../outputs/leituras_dutra_completo.csv', index=False)
df_dutra_kms_out

,Id,Rodovia,State,KM,esp_00h_06hs,esp_06_12hs,esp_12_18,esp_18_24hs
0,1,Presidente Dutra,SP,232,1,2,1,1
1,2,Presidente Dutra,SP,231,1,0,0,0
2,3,Presidente Dutra,SP,230,0,1,3,0
3,4,Presidente Dutra,SP,229,0,0,0,0
4,5,Presidente Dutra,SP,228,2,2,2,7
...,...,...,...,...,...,...,...,...
397,398,Presidente Dutra,RJ,169,0,0,0,0
398,399,Presidente Dutra,RJ,168,0,1,1,0
399,400,Presidente Dutra,RJ,167,1,4,2,3
400,401,Presidente Dutra,RJ,166,1,0,1,1
